#### Training 8 SVMs with Words

In [1]:
import nltk
import ssl, unicodedata, contractions

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('wordnet', quiet=True)
nltk.download('punkt', quiet=True)
nltk.download('omw-1.4', quiet=True)
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
import re
import string
import os
from nltk.stem import WordNetLemmatizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, cross_val_score
import nltk
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings("ignore")

In [2]:
files = os.listdir("./data/Postags/")
prefix_sentences = "./data/Sentences/"
prefix_label_A = "./data/Labels_A/"
prefix_label_CH = "./data/Labels_CH/"
prefix_label_CR = "./data/Labels_CR/"
prefix_label_J = "./data/Labels_J/"
prefix_label_LAW = "./data/Labels_LAW/"
prefix_label_LTD = "./data/Labels_LTD/"
prefix_label_TER = "./data/Labels_TER/"
prefix_label_USE = "./data/Labels_USE/"
prefix_label = "./data/Labels/"
    
word = []
for file in files:
    label_file_path_A = prefix_label_A + file
    label_file_path_CH = prefix_label_CH + file
    label_file_path_CR = prefix_label_CR + file
    label_file_path_J = prefix_label_J + file
    label_file_path_LAW = prefix_label_LAW + file
    label_file_path_LTD = prefix_label_LTD + file
    label_file_path_TER = prefix_label_TER + file
    label_file_path_USE = prefix_label_USE + file
    label_file_path = prefix_label + file
    sentences_file_path = prefix_sentences + file
    sentences_df = pd.read_csv(sentences_file_path, sep="dummy_separator", header=None)
    sentences_df.columns = ["sentence"]
        
    label_A_df = pd.read_csv(label_file_path_A, sep=" ", header=None, names=["label_A"])
    label_A_df["label_A_converted"] = np.where(label_A_df["label_A"] == -1, 0, 1)

    label_CH_df = pd.read_csv(label_file_path_CH, sep=" ", header=None, names=["label_CH"])
    label_CH_df["label_CH_converted"] = np.where(label_CH_df["label_CH"] == -1, 0, 1)

    label_CR_df = pd.read_csv(label_file_path_CR, sep=" ", header=None, names=["label_CR"])
    label_CR_df["label_CR_converted"] = np.where(label_CR_df["label_CR"] == -1, 0, 1)

    label_J_df = pd.read_csv(label_file_path_J, sep=" ", header=None, names=["label_J"])
    label_J_df["label_J_converted"] = np.where(label_J_df["label_J"] == -1, 0, 1)

    label_LAW_df = pd.read_csv(label_file_path_LAW, sep=" ", header=None, names=["label_LAW"])
    label_LAW_df["label_LAW_converted"] = np.where(label_LAW_df["label_LAW"] == -1, 0, 1)

    label_LTD_df = pd.read_csv(label_file_path_LTD, sep=" ", header=None, names=["label_LTD"])
    label_LTD_df["label_LTD_converted"] = np.where(label_LTD_df["label_LTD"] == -1, 0, 1)

    label_TER_df = pd.read_csv(label_file_path_TER, sep=" ", header=None, names=["label_TER"])
    label_TER_df["label_TER_converted"] = np.where(label_TER_df["label_TER"] == -1, 0, 1)

    label_USE_df = pd.read_csv(label_file_path_USE, sep=" ", header=None, names=["label_USE"])
    label_USE_df["label_USE_converted"] = np.where(label_USE_df["label_USE"] == -1, 0, 1)

    label_df = pd.read_csv(label_file_path, sep=" ", header=None, names=["label"])
    label_df["label_converted"] = np.where(label_df["label"] == -1, 0, 1)

    sentences_df["document"] = file
    df_concat = pd.concat([label_df["label_converted"], label_A_df["label_A_converted"], label_CH_df["label_CH_converted"], label_CR_df["label_CR_converted"],
                           label_J_df["label_J_converted"], label_LAW_df["label_LAW_converted"], label_LTD_df["label_LTD_converted"],
                           label_TER_df["label_TER_converted"], label_USE_df["label_USE_converted"], sentences_df[["sentence", "document"]]], axis=1)
    word.append(df_concat)

In [3]:
colnames = ["label_converted", "label_A_converted", "label_CH_converted", "label_CR_converted", "label_J_converted", "label_LAW_converted", "label_LTD_converted", "label_TER_converted", "label_USE_converted", "sentence", "document"]
clauses_df = pd.DataFrame(columns = colnames)
for df in word:
    clauses_df = clauses_df.append(df)

In [4]:
clauses_df.rename(columns={'label_converted': 'label', 'label_A_converted': 'label_A', 'label_CH_converted': 'label_CH', 'label_CR_converted': 'label_CR', 
                           'label_J_converted': 'label_J', 'label_LAW_converted': 'label_LAW', 'label_LTD_converted': 'label_LTD', 
                           'label_TER_converted': 'label_TER', 'label_USE_converted': 'label_USE',
                           'sentence': 'sentences', 'document' : 'document'}, inplace=True)
clauses_df.to_csv("data/svm8_word_merged.csv", index = False)

In [5]:
clauses_df.head()

,label,label_A,label_CH,label_CR,label_J,label_LAW,label_LTD,label_TER,label_USE,sentences,document
0,0,0,0,0,0,0,0,0,0,thanks for sending us good vibes by using the ...,Viber.txt
1,0,0,0,0,0,0,0,0,0,"you may be surprised , but we will refer to al...",Viber.txt
2,0,0,0,0,0,0,0,0,0,"the terms of use -lrb- or , the `` terms '' -r...",Viber.txt
3,0,0,0,0,0,0,0,0,0,the language of the terms will seem legal -lrb...,Viber.txt
4,1,0,0,0,0,0,0,0,1,"when you use our services , in addition to enj...",Viber.txt


In [6]:
assert (clauses_df.isnull().sum().all() == 0)

In [7]:
def to_lower(data: pd.Series):
    return data.str.lower()

def remove_accented_characters(data: pd.Series):
    return data.apply(lambda x: unicodedata.normalize("NFKD", x).encode("ascii", "ignore").decode("utf-8", "ignore"))

def remove_html_encodings(data: pd.Series):
    return data.str.replace(r"\d+;", " ", regex=True)

def remove_html_tags(data: pd.Series):
    return data.str.replace(r"<[a-zA-Z]+\s?/?>", " ", regex=True)

def remove_url(data: pd.Series):
    return data.str.replace(r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+", " ", regex=True)

def remove_html_and_url(data: pd.Series):
    data.str.replace(r"\d+;", " ", regex=True)
    data.str.replace(r"<[a-zA-Z]+\s?/?>", " ", regex=True)
    data.str.replace(r"https?://([\w\-\._]+){2,}/[\w\-\.\-/=\+_\?]+", " ", regex=True)
    return data

def remove_extra_spaces(data: pd.Series):
    return data.str.replace(r"^\s*|\s\s*", " ", regex=True)
                     
def remove_non_alpha_characters(data: pd.Series):
    return data.str.replace(r"_+|\\|[^a-zA-Z0-9\s]", " ", regex=True)

def fix_contractions(data: pd.Series):
    def contraction_fixer(txt: str):
        return " ".join([contractions.fix(word) for word in txt.split()])
    return data.apply(contraction_fixer)

def remove_special_words(data: pd.Series):
    return data.str.replace(r"\-[^a-zA-Z]{3}\-", " ", regex=True)
                     
data_cleaning_pipeline = {
    "sentences": [
        to_lower,
        remove_special_words,
        remove_accented_characters,
        remove_html_encodings,
        remove_html_tags,
        remove_url,
        fix_contractions,
        remove_non_alpha_characters,
        remove_extra_spaces,
    ]
}

cleaned_data = clauses_df.copy()

for col, pipeline in data_cleaning_pipeline.items():
    temp_data = cleaned_data[col].copy()
    for func in pipeline:
        print(f"Starting: {func.__name__}")
        temp_data = func(temp_data)
        print(f"Ended: {func.__name__}")
    cleaned_data[col] = temp_data.copy()

Starting: to_lower
Ended: to_lower
Starting: remove_special_words
Ended: remove_special_words
Starting: remove_accented_characters
Ended: remove_accented_characters
Starting: remove_html_encodings
Ended: remove_html_encodings
Starting: remove_html_tags
Ended: remove_html_tags
Starting: remove_url
Ended: remove_url
Starting: fix_contractions
Ended: fix_contractions
Starting: remove_non_alpha_characters
Ended: remove_non_alpha_characters
Starting: remove_extra_spaces
Ended: remove_extra_spaces


In [8]:
cleaned_data.document.unique()

array(['Viber.txt', 'Nintendo.txt', 'Tinder.txt', 'Dropbox.txt',
       'Microsoft.txt', 'Betterpoints_UK.txt', 'Airbnb.txt',
       'musically.txt', 'Crowdtangle.txt', 'TripAdvisor.txt',
       'Deliveroo.txt', 'Moves-app.txt', 'Spotify.txt', 'Supercell.txt',
       '9gag.txt', 'Booking.txt', 'Headspace.txt', 'Fitbit.txt',
       'Syncme.txt', 'Vimeo.txt', 'Oculus.txt', 'Endomondo.txt',
       'Instagram.txt', 'LindenLab.txt', 'WorldOfWarcraft.txt',
       'YouTube.txt', 'Academia.txt', 'Yahoo.txt', 'WhatsApp.txt',
       'Google.txt', 'Zynga.txt', 'Facebook.txt', 'Amazon.txt',
       'Vivino.txt', 'Netflix.txt', 'PokemonGo.txt', 'Skype.txt',
       'Snap.txt', 'eBay.txt', 'Masquerade.txt', 'Twitter.txt',
       'LinkedIn.txt', 'Skyscanner.txt', 'Duolingo.txt', 'TrueCaller.txt',
       'Uber.txt', 'Rovio.txt', 'Atlas.txt', 'Evernote.txt', 'Onavo.txt'],
      dtype=object)

In [9]:
cleaned_data[['sentences']]

,sentences
0,thanks for sending us good vibes by using the...
1,you may be surprised but we will refer to all...
2,the terms of use lrb or the terms rrb present...
3,the language of the terms will seem legal lrb...
4,when you use our services in addition to enjo...
...,...
142,the failure of onavo to enforce any right or ...
143,the section headings in the agreement are inc...
144,including whether capitalized or not means wi...
145,this agreement may not be assigned by you wit...


In [10]:
logo = LeaveOneGroupOut()
X = cleaned_data['sentences']
y = cleaned_data[['label', 'label_A', 'label_CH', 'label_CR', 'label_J', 'label_LAW', 'label_LTD', 'label_TER', 'label_USE']]
group = cleaned_data['document']
logo.get_n_splits(X, y, group)

50

In [11]:
train_val_test = []
for train_val_index, test_index in logo.split(X, y, group):
    train_val, test = cleaned_data.iloc[train_val_index], cleaned_data.iloc[test_index]
    train_val_test.append((train_val, test))

In [12]:
def train_svm(x_batch, y_batch, label, nrange):
    X_train = x_batch["sentences"]
    y_train = x_batch[label]
    train_groups = x_batch["document"]
    X_test = y_batch["sentences"]
    y_test = y_batch[label]
    test_document = y_batch.document.unique()[0]
    vectorizer = TfidfVectorizer(lowercase = True, ngram_range = nrange)
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)
    y_train = y_train.astype('int')
    y_test = y_test.astype('int')
    
    svm = LinearSVC(random_state=0, max_iter = 5000)
    Cs = [0.001, 0.01, 0.1, 1, 10]
    clf = GridSearchCV(estimator=svm, param_grid=dict(C=Cs),n_jobs=-1, scoring = 'f1', refit = True)
    clf_fit = clf.fit(X_train, y_train, groups = train_groups)
    clf_best = clf_fit.best_estimator_
    y_test_pred = clf_best.predict(X_test)
    score = clf.score(X_test, y_test)
    return y_test_pred, score

In [13]:
ngram_ranges = [(1,1), (1,2), (2,2), (1,3), (2,3), (3,3)]
scores_compare = {}
y_test_pred_A = {}
y_test_pred_CH = {}
y_test_pred_CR = {}
y_test_pred_J = {}
y_test_pred_LAW = {}
y_test_pred_LTD = {}
y_test_pred_TER = {}
y_test_pred_USE = {}
y_test_all = {}
for nrange in ngram_ranges:
    print('Training ngram_range: ', nrange)
    scores_compare[nrange] = {}
    scores_A = []
    scores_CH = []
    scores_CR = []
    scores_J = []
    scores_LAW = []
    scores_LTD = []
    scores_TER = []
    scores_USE = []
    y_test_pred_A[nrange] = []
    y_test_pred_CH[nrange] = []
    y_test_pred_CR[nrange] = []
    y_test_pred_J[nrange] = []
    y_test_pred_LAW[nrange] = []
    y_test_pred_LTD[nrange] = []
    y_test_pred_TER[nrange] = []
    y_test_pred_USE[nrange] = []
    y_test_all[nrange] = []
    for batch in train_val_test:
        ## Train on Label A
        y_test_pred, score = train_svm(batch[0], batch[1], 'label_A', nrange)
        y_test_pred_A[nrange].extend(y_test_pred)
        scores_A.append(score)

        ## Train on Label CH
        y_test_pred, score = train_svm(batch[0], batch[1], 'label_CH', nrange)
        y_test_pred_CH[nrange].extend(y_test_pred)
        scores_CH.append(score)

        ## Train on Label CR
        y_test_pred, score = train_svm(batch[0], batch[1], 'label_CR', nrange)
        y_test_pred_CR[nrange].extend(y_test_pred)
        scores_CR.append(score)

        ## Train on Label J
        y_test_pred, score = train_svm(batch[0], batch[1], 'label_J', nrange)
        y_test_pred_J[nrange].extend(y_test_pred)
        scores_J.append(score)

        ## Train on Label LAW
        y_test_pred, score = train_svm(batch[0], batch[1], 'label_LAW', nrange)
        y_test_pred_LAW[nrange].extend(y_test_pred)
        scores_LAW.append(score)

        ## Train on Label LTD
        y_test_pred, score = train_svm(batch[0], batch[1], 'label_LTD', nrange)
        y_test_pred_LTD[nrange].extend(y_test_pred)
        scores_LTD.append(score)

        ## Train on Label TER
        y_test_pred, score = train_svm(batch[0], batch[1], 'label_TER', nrange)
        y_test_pred_TER[nrange].extend(y_test_pred)
        scores_TER.append(score)

        ## Train on Label USE
        y_test_pred, score = train_svm(batch[0], batch[1], 'label_USE', nrange)
        y_test_pred_USE[nrange].extend(y_test_pred)
        scores_USE.append(score)

        ## Getting all ground truth labels
        y_test_all[nrange].extend(batch[1]["label"])
        
    scores_compare[nrange]['A'] = sum(scores_A)/len(scores_A)
    scores_compare[nrange]['CH'] = sum(scores_CH)/len(scores_CH)
    scores_compare[nrange]['CR'] = sum(scores_CR)/len(scores_CR)
    scores_compare[nrange]['J'] = sum(scores_J)/len(scores_J)
    scores_compare[nrange]['LAW'] = sum(scores_LAW)/len(scores_LAW)
    scores_compare[nrange]['LTD'] = sum(scores_LTD)/len(scores_LTD)
    scores_compare[nrange]['TER'] = sum(scores_TER)/len(scores_TER)
    scores_compare[nrange]['USE'] = sum(scores_USE)/len(scores_USE)

Training ngram_range:  (1, 1)
Training ngram_range:  (1, 2)
Training ngram_range:  (2, 2)
Training ngram_range:  (1, 3)
Training ngram_range:  (2, 3)
Training ngram_range:  (3, 3)


In [14]:
scores_compare

{(1, 1): {'A': 0.2866666666666667,
  'CH': 0.6361962481962484,
  'CR': 0.5312121212121214,
  'J': 0.6258484848484849,
  'LAW': 0.7746666666666667,
  'LTD': 0.6395404903340388,
  'TER': 0.6532168811580576,
  'USE': 0.7329437229437231},
 (1, 2): {'A': 0.30166666666666664,
  'CH': 0.6731016250303179,
  'CR': 0.5403376623376624,
  'J': 0.577038961038961,
  'LAW': 0.7880000000000001,
  'LTD': 0.6853640216939451,
  'TER': 0.654110360743488,
  'USE': 0.7510476190476192},
 (2, 2): {'A': 0.255,
  'CH': 0.6396349206349209,
  'CR': 0.3990995670995672,
  'J': 0.5566580086580087,
  'LAW': 0.7966666666666667,
  'LTD': 0.7042249088395099,
  'TER': 0.5899481074481078,
  'USE': 0.6952380952380954},
 (1, 3): {'A': 0.295,
  'CH': 0.6575180375180376,
  'CR': 0.502909090909091,
  'J': 0.6258095238095239,
  'LAW': 0.7946666666666666,
  'LTD': 0.7090292083591317,
  'TER': 0.6463922618489184,
  'USE': 0.754952380952381},
 (2, 3): {'A': 0.26476190476190475,
  'CH': 0.6296445406445409,
  'CR': 0.365766233766233

### Combine predictions

In [15]:
y_pred = {}
for nrange in ngram_ranges:
    y_pred[nrange] = []
    for idx in range(len(y_test_all[nrange])):
        if y_test_pred_A[nrange][idx] == 1 or y_test_pred_CH[nrange][idx] == 1 or y_test_pred_CR[nrange][idx] == 1 or y_test_pred_J[nrange][idx] == 1 or y_test_pred_LTD[nrange][idx] == 1 or y_test_pred_TER[nrange][idx] == 1 or y_test_pred_USE[nrange][idx] == 1:
            y_pred[nrange].append(1) ## if anyone model predicts clause as unfair then it is unfair
        else:
            y_pred[nrange].append(0)

In [16]:
for nrange in ngram_ranges:    
    print('Ngram range: ', nrange)
    report = classification_report(y_test_all[nrange], y_pred[nrange])
    print(report)

Ngram range:  (1, 1)
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      8382
           1       0.83      0.60      0.70      1032

    accuracy                           0.94      9414
   macro avg       0.89      0.79      0.83      9414
weighted avg       0.94      0.94      0.94      9414

Ngram range:  (1, 2)
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      8382
           1       0.86      0.62      0.72      1032

    accuracy                           0.95      9414
   macro avg       0.91      0.80      0.84      9414
weighted avg       0.94      0.95      0.94      9414

Ngram range:  (2, 2)
              precision    recall  f1-score   support

           0       0.95      0.99      0.97      8382
           1       0.86      0.58      0.69      1032

    accuracy                           0.94      9414
   macro avg       0.91      0.78      0.83      9414
weighted av

ngram range of (1, 3) i.e. unigrams, bigrams, trigrams and combinations gives the best macro average F1-score for combined SVM.